<a href="https://colab.research.google.com/github/shinyabe/-/blob/main/%E5%8D%92%E8%AB%96%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np  # 数値計算ライブラリ
import pandas as pd  # データフレーム操作用
import itertools  # 全戦略の組み合わせ生成に使用
import random  # 乱数生成

# 定数（シミュレーション日数や消費者需要）※消費者数は正規分布に従う
DAYS = 30  # シミュレーション日数
CONSUMER_MEAN = 100  # 1日の平均来店者数
CONSUMER_STD = 20  # 1日の来店者数の標準偏差
VARIATION_COEFF = CONSUMER_STD / CONSUMER_MEAN  # 消費者数の変動係数

# 食材の劣化パラメータ設定
MU1, MU2 = 5, 2  # 状態1→2および状態2→3の平均時間
S11, S22 = 2, 1  # 状態1→2および状態2→3の時間の分散
RHO = 0.8  # 相関係数
#卸売業者の目利きコスト戦略が食材の平均寿命（t1,t2の平均）と分散に与える影響度合いの設定
ALPHA_X = 1.2  # 目利きコストによる平均寿命の拡張倍率
ALPHA_Y = 0.8  # 目利きコストによる分散縮小倍率
#卸売業者と小売業者の保存方法戦略が、食材の鮮度低下速度に与える影響度合いを設定
BETA_X, BETA_Y = 0.5, 1.0  # 高品質保存と低価格保存の劣化速度

# 消費者が購入する数の最大値（仮に設定）
MAX_PURCHASE_PER_CUSTOMER = 3

# 卸売・小売の戦略の全パターン生成
#卸売業者の戦略
wholesaler_strategies = list(itertools.product(
    ['high', 'low'],  # 安全在庫量（多め or 少なめ）
    ['high', 'low'],  # 目利きコスト（かける or かけない）
    ['high', 'low'],  # 冷蔵保存（高品質 or 低価格）
    ['high', 'low'],  # エチレン除去（未使用）
    ['FIFO', 'LIFO']  # 出荷順（先入先出 or 後入先出）
))

#小売店の戦略
retailer_strategies = list(itertools.product(
    ['high', 'none'],  # 割引戦略（有 or 無）
    ['high', 'low']   # 冷蔵保存戦略
))

# 状態遷移時間の生成（2次元正規分布）
# 状態1→2および状態2→3の平均時間（mu1,mu2）、と分散（s11,s22）、相関係数（rho）を持つ2次元正規分布から食材の劣化に関する2つの時間パラメータt1,t2を生成
def generate_lifetimes(mu1, mu2, s11, s22, rho):
    cov = [[s11, rho * np.sqrt(s11 * s22)], [rho * np.sqrt(s11 * s22), s22]]  # 共分散行列
    return np.random.multivariate_normal([mu1, mu2], cov)  # t1, t2を生成

# 鮮度状態の判定関数（状態1,2,3）
#食材の生産日（product_day）、現在の日付(today)、食材固有の劣化時間(t1,t2)、および鮮度低下速度(fd)を使用して、食材の腐敗進捗度(fp)を算出
def determine_state(produced_day, today, t1, t2, fd):
    fp = -fd * (today - produced_day) + t1 + t2  # 腐敗進捗度
    if fp > t2:
        return 1  # 状態1（良品）
    elif fp > 0:
        return 2  # 状態2（スレ傷あり）
    else:
        return 3  # 状態3（腐敗）

# 消費者が購入するかの判断（状態と値引き）
def consumer_decides(state, discounted):  #食材の状態(state)と小売店が割引を行っているかの有無(discounted)
    if state == 1: #状態1であれば購入
        return True
    elif state == 2 and discounted: #状態2では割引を行なっている場合にのみ購入
        return True
    return False #状態3の食材は購入されない

# 1戦略のシミュレーション実行関数
def run_simulation(wh_strategy, ret_strategy): #卸売業者 (wh_strategy) と小売業者 (ret_strategy) の戦略の組み合わせを受け取る
    total_r_profit = 0  # 小売利益
    total_discard = 0  # 廃棄量
    inventory = []  # 小売在庫

    for day in range(DAYS):
        # 卸の食材納入
        mu1, mu2 = MU1, MU2
        s11, s22 = S11, S22
        fd = BETA_X if wh_strategy[2] == 'high' else BETA_Y  # 卸の保存方式による腐敗速度

        # 目利き戦略が高い場合、平均を延ばし分散を縮小
        if wh_strategy[1] == 'high':
            mu1 *= ALPHA_X
            mu2 *= ALPHA_X
            s11 *= ALPHA_Y
            s22 *= ALPHA_Y

        # 卸売業者の安全在庫量戦略（未使用ですが、将来的な拡張のために残します）
        # if wh_strategy[0] == 'high':
        #     pass # 多めに納入するロジックをここに追加
        # else:
        #     pass # 少なめに納入するロジックをここに追加

        t1, t2 = generate_lifetimes(mu1, mu2, s11, s22, RHO)
        inventory.append({'produced_day': day, 't1': t1, 't2': t2, 'fd': fd}) #小売在庫 (inventory) に追加

        # 消費者数を正規分布からサンプリング
        customer_num = int(np.random.normal(CONSUMER_MEAN, CONSUMER_STD)) #消費者数をランダムに生成
        sold = 0 #販売数の初期値
        discarded = 0 #廃棄量の初期値

        # 小売の戦略設定
        discount = ret_strategy[0] == 'high' # 小売戦略の0番目が割引戦略になりました
        cold_fd = BETA_X if ret_strategy[1] == 'high' else BETA_Y # 小売戦略の1番目が冷蔵保存戦略になりました

        # 出荷順序（FIFO or LIFO）
        inventory = sorted(inventory, key=lambda x: x['produced_day'], reverse=(wh_strategy[4] == 'LIFO'))

        # 購入処理（1人ランダムな数を購入）
        for _ in range(customer_num):
            # 各消費者が購入する数をランダムに決定（1個からMAX_PURCHASE_PER_CUSTOMER個まで）
            items_to_buy = random.randint(1, MAX_PURCHASE_PER_CUSTOMER)
            bought_count = 0
            # 在庫を順番に見て、購入可能な商品をitems_to_buyの数だけ探す
            # 在庫リストをコピーしてイテレーション中に削除できるようにする
            for item in inventory[:]:
                if bought_count < items_to_buy:
                    state = determine_state(item['produced_day'], day, item['t1'], item['t2'], item['fd'] + cold_fd)
                    if consumer_decides(state, discount):
                        sold += 1
                        inventory.remove(item)
                        bought_count += 1
                else:
                    # 必要な数を購入したらこの消費者については次の商品を探すのをやめる
                    break


        # 腐敗品の廃棄処理
        # 在庫リストをコピーしてイテレーション中に削除できるようにする
        for item in inventory[:]:
            state = determine_state(item['produced_day'], day, item['t1'], item['t2'], item['fd'] + cold_fd)
            if state == 3:
                inventory.remove(item)
                discarded += 1

        revenue = sold * (120 if discount else 150)  # 単価設定 (割引あり120, なし150)
        # 在庫コスト：その日の終わりに残っている在庫数 * 1個あたりの保管コスト（例: 10）
        # 廃棄コスト：その日に廃棄された数 * 1個あたりの廃棄にかかるコスト（例: 50）
        cost = len(inventory) * 10 + discarded * 50
        total_r_profit += revenue - cost
        total_discard += discarded

    return total_r_profit, total_discard

# 全組合せでシミュレーション実行
results = []
# 小売戦略のタプル構造が変わったため、ここで調整します
# 以前: (安全在庫量, 割引戦略, 冷蔵保存戦略)
# 修正後: (割引戦略, 冷蔵保存戦略)
# 卸売戦略のタプル構造は変わりません
# (安全在庫量, 目利きコスト, 冷蔵保存, エチレン除去, 出荷順)




for w in wholesaler_strategies:
    for r in retailer_strategies:
        # 小売戦略のタプル構造変更に伴い、run_simulationに渡す際に調整が必要ですが、
        # retailer_strategiesの定義自体を変更したので、そのまま渡せます。
        profit, discard = run_simulation(w, r)
        results.append({
            '卸戦略': w,
            '小売戦略': r,
            '小売利益': profit,
            '廃棄量': discard
        })

# DataFrameで結果出力
df_results = pd.DataFrame(results)
print(df_results)  # 結果表示（利益・廃棄量）

                                卸戦略          小売戦略  小売利益  廃棄量
0    (high, high, high, high, FIFO)  (high, high)  3600    0
1    (high, high, high, high, FIFO)   (high, low)  3600    0
2    (high, high, high, high, FIFO)  (none, high)  4500    0
3    (high, high, high, high, FIFO)   (none, low)  4500    0
4    (high, high, high, high, LIFO)  (high, high)  3600    0
..                              ...           ...   ...  ...
123      (low, low, low, low, FIFO)   (none, low)  4500    0
124      (low, low, low, low, LIFO)  (high, high)  3600    0
125      (low, low, low, low, LIFO)   (high, low)  3600    0
126      (low, low, low, low, LIFO)  (none, high)  4500    0
127      (low, low, low, low, LIFO)   (none, low)  4500    0

[128 rows x 4 columns]
